In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import unidecode
import os

In [2]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

#### Funciones

In [3]:
def doClean(a):   
    a = str(a)
    # Quitar las expresiones tras la barra /
    a = re.sub('\/.*', '', a)
    # Quitar las aclaraciones entre paréntesis
    a = re.sub('\(.*\)', '', a)
    # Quitar acentos 
    a = unidecode.unidecode(a)
    # Quitar aquello que no sean palabras o cosa que se le parezca
    a = re.sub('\W', ' ', a) 
    # Quitar espacios extra en caso de haber
    a = re.sub('\s+', ' ', a)
    # Pasar el texto a minúsulas
    a = a.lower() 
    a = a.lstrip()
    # Quitar espacio derecha
    a = a.rstrip()
    a = re.sub(' ', '_', a)
    #a = a.strip() 
    
    return a

In [4]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=1):
    """
    df_1 is the left table to join
    df_2 is the right table to join
    key1 is the key column of the left table
    key2 is the key column of the right table
    threshold is how close the matches should be to return a match, based on Levenshtein distance
    limit is the amount of matches that will get returned, these are sorted high to low
    """
    s = df_2[key2].tolist()

    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m

    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2

    return df_1

In [138]:
def union_muni(df_maestro, df_union, muni_maestro, agrup_maestro, muni_union, agrup_union, threshold=85, threshold_mun=90):
    
    count_maestro = df_maestro.shape[0]
    count_union = df_union.shape[0]
    print('num registros maestro: {}'.format(count_maestro))
    print('num registros fichero para unir: {}'.format(count_union))
    
    #fuzzy por prov/ccaa
    maestro_agrup = pd.DataFrame(df_maestro[agrup_maestro].drop_duplicates())
    print('num agrupaciones maestro: {}'.format(maestro_agrup.shape))
    union_agrup = pd.DataFrame(df_union[agrup_union].drop_duplicates())
    print('num agrupaciones union: {}'.format(union_agrup.shape))

    agrup_match = fuzzy_merge(maestro_agrup, union_agrup, agrup_maestro, agrup_union, threshold=threshold)
    print('num agrupaciones tras match: {}'.format(agrup_match.shape))
    num_matches = (agrup_match[agrup_match['matches']!='']['matches'].nunique())
    if (num_matches != agrup_match[agrup_maestro].nunique()):
        print('no se han cruzado todos los valores:')
        print(agrup_match[agrup_match['matches']==''])

        #se genera un nuevo df con el match y el fichero de union
    df_union_agrup_match = pd.merge(df_union, agrup_match, left_on=agrup_union, right_on='matches', how='left')
    if (count_union != df_union_agrup_match.shape[0]):
        print('el fichero de union no dispone de todos los valores para cruzar')    

    df_union_agrup_match.drop(columns=['matches'], inplace=True)
    
    #fuzzy por municipio
    df_maestro['muni_agrup'] = df_maestro[muni_maestro]+"/"+df_maestro[agrup_maestro]
    df_union_agrup_match['muni_agrup'] = df_union_agrup_match[muni_union]+"/"+df_union_agrup_match[agrup_maestro]
   
    muni_match = pd.DataFrame()
    maestro_count = 0
    for agrup in list(maestro_agrup[agrup_maestro]):
        print(agrup)
        maestro_agrup_sel = df_maestro[df_maestro[agrup_maestro] == agrup].copy()
        maestro_count = maestro_agrup_sel.shape[0] + maestro_count
        union_agrup_sel = df_union_agrup_match[df_union_agrup_match[agrup_maestro] == agrup].copy()
        if union_agrup_sel.empty == True:
            print(f'no existe para {agrup}')
        
        else:
            aux = fuzzy_merge(maestro_agrup_sel, union_agrup_sel, 'muni_agrup', 'muni_agrup', threshold=threshold_mun)
            num_matches = (aux[aux['matches']!='']['matches'].nunique())
            if (num_matches != aux[muni_maestro].nunique()):
                print('no se han cruzado todos los valores:')
                #print(aux[aux['matches'].isnull()]) 
            muni_match = pd.concat([muni_match, aux])
        
        #comprobacion registros    
    print('num muni maestro: {}'.format(maestro_count))
    if maestro_count != muni_match.shape[0]:
        print("se han perdido registros")       
        
    muni_match.drop(columns='muni_agrup', inplace=True)
    df_union_agrup_match.drop(columns=[agrup_maestro], inplace=True)

        #se genera un nuevo df con el match y el fichero de union
    df_union_muni_match = pd.merge(muni_match, df_union_agrup_match, left_on='matches', right_on='muni_agrup', how='left')
    if (count_union != df_union_muni_match.shape[0]):
        print('el fichero de union no dispone de todos los valores para cruzar por municipio')        

    #merge maestro con fichero union
    #df_tras_union = pd.merge(df_maestro, df_union_muni_match, left_on=muni_maestro, right_on='matches', how='left')
    
    print('columnas final: {}'.format(df_union_muni_match.columns))
        #borramos columnas que sobran
    if muni_maestro != muni_union:
        df_union_muni_match.drop(columns=muni_union, inplace=True)
    if agrup_maestro != agrup_union:
        df_union_muni_match.drop(columns=agrup_union, inplace=True)
    df_union_muni_match.drop(columns=['matches', 'muni_agrup'], inplace=True)
    print('num registros tras union: {}'.format(df_union_muni_match.shape))
    
    return df_union_muni_match

#### Carga y union de ficheros

In [6]:
os.getcwd()

'/home/patcalsi/projects/Exodus_urbano/data/patri'

In [7]:
os.listdir()

['sociedad_seguridad_guardia_civil_2019_limpio.csv',
 'maestro_nivel_municipio_ori.csv',
 'union_ficheros.ipynb',
 'sociedad_educacion_universidad_2018_limpio.csv',
 'maestro_nivel_cod_postal.csv',
 'union_ficheros2.ipynb',
 'sociedad_sanidad_hospitales_2019_limpio.csv',
 'union_patri.csv',
 'sociedad_educacion_colegios_2020_limpio.csv',
 'sociedad_sanidad_centros_salud_2019_limpio.csv',
 'maestro_nuevo.csv',
 'maestro_nivel_municipio.csv',
 '.ipynb_checkpoints',
 '.~lock.Empresas-SICTED2.xls#',
 'economia_turismo_establecimientos_20XX_limpio.csv',
 '.~lock.union_patri.csv#',
 'tratamiento',
 'sociedad_usos_suelo_2019_limpio.csv',
 'manual.txt',
 'aaaa.csv',
 'sociedad_cultura_bibliotecas_2017_limipio.csv',
 'listado-codigos-postales-con-LatyLon.csv',
 'sociedad_educacion_nivel_formacion_ccaa_2018.csv']

## Fichero maestro

In [173]:
maestro_municipios = pd.read_csv("../maestro_idiomas_ine.csv", index_col=0, dtype={'postal_code': object, 'ine_code':object})
maestro_municipios.shape

(14729, 17)

In [174]:
#correcciones
maestro_municipios['province'].replace("islas_baleares", "illes_balears", inplace=True )
maestro_municipios['cities_es/province'].replace("islas_baleares", "illes_balears", inplace=True, regex=True)
maestro_municipios['ccaa'].replace("islas_baleares", "illes_balears", inplace=True, regex=True)
maestro_municipios['ccaa'].replace("comunidad_valenciana", "comunitat_valenciana", inplace=True, regex=True)
maestro_municipios['province'].replace("guipuzcoa", "gipuzkoa", inplace=True )
#maestro_municipios['cities_es/province'].replace("guipuzcoa", "gipuzkoa", inplace=True, regex=True)

#eliminacion nans
maestro_municipios = maestro_municipios[~maestro_municipios['province'].isnull()]

In [175]:
maestro_municipios.head(2)

,ine_code,postal_code,cities_es,cities,province_es,province,ccaa_es,ccaa,cities_es/province_es,cities_es/province,cities_es/ccaa_es,cities_es/ccaa,cities/province_es,province_es/ccaa_es,province_es/ccaa,province/ccaa_es,province/ccaa
0,01001,01240,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco
1,01001,01193,alegria_dulantzi,alegria_dulantzi,alava,araba,pais_vasco,pais_vasco,alegria_dulantzi/alava,alegria_dulantzi/araba,alegria_dulantzi/pais_vasco,alegria_dulantzi/pais_vasco,alegria_dulantzi/alava,alava/pais_vasco,alava/pais_vasco,araba/pais_vasco,araba/pais_vasco


In [176]:
maestro_municipios.shape

(14288, 17)

In [180]:
maestro_municipios.to_csv("maestro_nivel_todo.csv")

### Creacion maestro nivel cod postal

In [12]:
# se eliminan el ine y cod postal
maestro_nivel_cod_postal = maestro_municipios.drop(columns=['ine_code','cities_es/province_es',
       'cities_es/ccaa_es', 'cities_es/ccaa', 'cities_es', 'cities', 'province_es', 'province',
       'ccaa_es', 'ccaa', 'cities/province_es', 'province_es/ccaa_es', 
        'province_es/ccaa', 'province/ccaa_es', 'province/ccaa'])
maestro_nivel_cod_postal.drop_duplicates(inplace=True)
maestro_nivel_cod_postal.shape

(10672, 2)

In [13]:
maestro_nivel_cod_postal.columns

Index(['postal_code', 'cities_es/province'], dtype='object')

In [14]:
maestro_nivel_cod_postal.to_csv("maestro_nivel_cod_postal.csv")

### Creacion maestro nivel municipio

In [15]:
# se eliminan el ine y cod postal
maestro_nivel_municipio = maestro_municipios.drop(columns=['ine_code','postal_code', 'cities_es/province_es', 'cities_es/ccaa_es',
       'cities_es/ccaa', 'cities/province_es', 'province_es/ccaa_es',
       'province_es/ccaa', 'province/ccaa_es', 'province/ccaa'])
maestro_nivel_municipio.drop_duplicates(inplace=True)
maestro_nivel_municipio.shape

(6067, 7)

In [16]:
maestro_nivel_municipio.columns

Index(['cities_es', 'cities', 'province_es', 'province', 'ccaa_es', 'ccaa',
       'cities_es/province'],
      dtype='object')

In [17]:
maestro_nivel_municipio.to_csv("maestro_nivel_municipio.csv")

### Fichero nombres originales

### Creacion maestro nivel cod INE

In [181]:
# se eliminan el ine y cod postal
maestro_nivel_ine = maestro_municipios.drop(columns=['postal_code','cities_es/province_es',
       'cities_es/ccaa_es', 'cities_es/ccaa', 'cities_es', 'cities', 'province_es', 'province',
       'ccaa_es', 'ccaa', 'cities/province_es', 'province_es/ccaa_es', 
        'province_es/ccaa', 'province/ccaa_es', 'province/ccaa'])
maestro_nivel_ine.drop_duplicates(inplace=True)
maestro_nivel_ine.shape

(9511, 2)

In [182]:
maestro_nivel_ine.columns

Index(['ine_code', 'cities_es/province'], dtype='object')

In [185]:
maestro_nivel_ine[maestro_nivel_ine.duplicated()]

,ine_code,cities_es/province


In [186]:
maestro_nivel_ine.to_csv("maestro_nivel_ine.csv")